In [15]:
import json
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import math

In [16]:
#os.getcwd()
os.chdir("/Users/zaio/dev/time-posture-analysis-tool")
# os.chdir(r"C:\Users\ZAIO\dev\time-posture-analysis-tool")

# %%
parent_dir = "./data_reconstruction"

json_dir  = os.path.join(parent_dir,"json")
img_dir  = os.path.join(parent_dir,"img")

img_list  = os.listdir(img_dir)
img_list.sort()


json_list  = os.listdir(json_dir)
json_list.sort()

xy_pos_data = []
for i in range(3):
    file_name = os.path.join(json_dir,json_list[i])
    with open(file_name) as json_file:
        data = json.load(json_file)
        xy_pos_data.append(data['people'][0]['pose_keypoints_2d'][8*3:8*3+2])

file_name = os.path.join(json_dir,json_list[i])

with open(file_name) as json_file:
    data = json.load(json_file)

xyz_pos_data_joints  = data['people'][0]['pose_keypoints_3d']
xyz_pos_data = data['people'][0]['pose_keypoints_3d'][8*4:8*4+3]

In [17]:
#load xml file
camera_parameters = {}
parameter_dir=r"C:\Users\ZAIO\dev\openpose\openpose\models\cameraParameters\flir\20200810"
camera_idxs = ["18284509","18284511"]
parameter_types = ["CameraMatrix","Intrinsics","Distortion"]


for camera_idx in camera_idxs:
    fileToLoad = os.path.join(parameter_dir,camera_idx+".xml")
    fs = cv2.FileStorage(fileToLoad, cv2.FILE_STORAGE_READ)
    _camera_parameter = {}
    for parameter_type in parameter_types:
        _camera_parameter[parameter_type] = fs.getNode(parameter_type).mat()
        camera_parameters[camera_idx] = _camera_parameter

In [18]:
def undistort_points(points2d,camera_index):
        K = intri[camera_index]
        distCoef = dist[camera_index]

        """
        :param points2d: [ (x,y,w), ...]
        :return:
        """
        points2d_ = points2d[:, 0:2].astype('float32')
        points2d_ = np.expand_dims(points2d_, axis=1)  # (n, 1, 2)
        result = np.squeeze(cv2.undistortPoints(points2d_, K, distCoef))

        if len(result.shape) == 1:  # only a single point
            result = np.expand_dims(result, 0)

        fx = K[0, 0]
        fy = K[1, 1]
        cx = K[0, 2]
        cy = K[1, 2]

        points2d_undist = np.empty_like(points2d)
        for i, (px, py) in enumerate(result):
            points2d_undist[i, 0] = px * fx + cx
            points2d_undist[i, 1] = py * fy + cy
            points2d_undist[i, 2] = points2d[i, 2]

        return points2d_undist 
    
def undistortPoints(_x,_y,cameraIdx = i):
    points2d  = np.array([(_x,_y)])
    points2d = np.float32(points2d[:, np.newaxis, :])
    x =  cv2.undistortPoints(points2d,intri[i],dist[i])[0][0][0]
    y =  cv2.undistortPoints(points2d,intri[i],dist[i])[0][0][1]
    return x,y


#3d reconstruction using 1st and 2nd camera
def triangulatePoints(cam01_x,cam01_y,cam02_x,cam02_y,cam01_idx,cam02_idx):

    undistort_x_0 = undistort_points(np.array([(cam01_x,cam01_y,1)]),cam01_idx)[0][0]
    undistort_y_0 = undistort_points(np.array([(cam01_x,cam01_y,1)]),cam01_idx)[0][1]


    undistort_x_1 = undistort_points(np.array([(cam02_x,cam02_y,1)]),cam02_idx)[0][0]
    undistort_y_1 = undistort_points(np.array([(cam02_x,cam02_y,1)]),cam02_idx)[0][1]

    tri_array = cv2.triangulatePoints(projMatr[cam01_idx],projMatr[cam02_idx],(undistort_x_0,undistort_y_0),(undistort_x_1,undistort_y_1))
    tri_xyz = tri_array/tri_array[3]
    return np.reshape(tri_xyz,-1)[0:3]


In [21]:
#3d points to image (2nd image)
extri = [0]*3
intri  = [0]*3
dist = [0]*3

for i in range(len(camera_idxs)):
    rotation_matrix = camera_parameters[camera_idxs[i]][parameter_types[0]][:,0:3]
    rvec,_ = cv2.Rodrigues(rotation_matrix)
    tvec = camera_parameters[camera_idxs[i]][parameter_types[0]][:,3]
    
    extri[i] = camera_parameters[camera_idxs[i]][parameter_types[0]]
    intri[i] = camera_parameters[camera_idxs[i]][parameter_types[1]]
    dist[i] = camera_parameters[camera_idxs[i]][parameter_types[2]]
    
#Triangulation
projMatr = [0]*3
for i in range(len(camera_idxs)):  
    projMatr[i] = np.dot(intri[i],extri[i])
    
    
cam_idx = 0
_x = xy_pos_data[cam_idx][0]
_y = xy_pos_data[cam_idx][1]

undistort_x_0 = undistort_points(np.array([(_x,_y,1)]),cam_idx)[0][0]
undistort_y_0 = undistort_points(np.array([(_x,_y,1)]),cam_idx)[0][1]


cam_idx = 1
_x = xy_pos_data[cam_idx][0]
_y = xy_pos_data[cam_idx][1]

undistort_x_1 = undistort_points(np.array([(_x,_y,1)]),cam_idx)[0][0]
undistort_y_1 = undistort_points(np.array([(_x,_y,1)]),cam_idx)[0][1]

tri_array = cv2.triangulatePoints(projMatr[0],projMatr[1],(undistort_x_0,undistort_y_0),(undistort_x_1,undistort_y_1))
tri_xyz = tri_array/tri_array[3]




In [22]:
# Store the image as dataframe
# Undistort each value
# Run the 3d reconstruction

sofa_dir = os.path.join(parent_dir,"sofa_points.csv")
data = pd.read_csv(sofa_dir, index_col = "idx")

sofa_points = {}
xy = ["x","y"]

for i in range(len(data.index)):
    sofa_xy = {}
    for col_idx in data.columns:
        sofa_xy[col_idx] = data.loc[i][col_idx]
    sofa_points[i] = sofa_xy

    
_x01 = xy_pos_data[0][0]
_y01 = xy_pos_data[0][1]
_x02 = xy_pos_data[1][0]
_y02 = xy_pos_data[1][1]


In [23]:
sofa_dir = os.path.join(parent_dir,"sofa_points.csv")
data = pd.read_csv(sofa_dir, index_col = "idx")

In [24]:
data_tri = data.copy()

cam_idx_first = 0
cam_idx_second = 1
tri_00_01 = []

for i in data_tri.index:
    _x01 = float(data_tri.loc[i]["x"+str(cam_idx_first)])
    _y01 = float(data_tri.loc[i]["y"+str(cam_idx_first)])
    _x02 = float(data_tri.loc[i]["x"+str(cam_idx_second)])
    _y02 = float(data_tri.loc[i]["y"+str(cam_idx_second)])
    tri_00_01.append(triangulatePoints(_x01,_y01,_x02,_y02,cam_idx_first,cam_idx_second))
    

In [25]:
tri_00_01

[array([-1.27128908,  1.06387257,  5.60741768]),
 array([-0.75491387,  0.58966565,  4.57628397]),
 array([0.48666561, 0.34209647, 3.64867758]),
 array([0.7497733 , 0.51554642, 3.72946823]),
 array([-1.29676362,  0.63023679,  5.14691096]),
 array([-0.88119139,  0.27379779,  4.21591129]),
 array([-0.70344733,  0.05108803,  3.98848722]),
 array([ 0.47319009, -0.05143787,  3.24503007]),
 array([0.60185908, 0.0843284 , 3.33146798]),
 array([0.79640022, 0.25372233, 3.43892672])]

In [26]:
cam_idx_first = 1
cam_idx_second = 2
tri_01_02 = []


for i in data_tri.index:
    _x01 = float(data_tri.loc[i]["x"+str(cam_idx_first)])
    _y01 = float(data_tri.loc[i]["y"+str(cam_idx_first)])
    _x02 = float(data_tri.loc[i]["x"+str(cam_idx_second)])
    _y02 = float(data_tri.loc[i]["y"+str(cam_idx_second)])
    tri_01_02.append(triangulatePoints(_x01,_y01,_x02,_y02,cam_idx_first,cam_idx_second))
    

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\calib3d\src\undistort.dispatch.cpp:284: error: (-215:Assertion failed) CV_IS_MAT(_cameraMatrix) && _cameraMatrix->rows == 3 && _cameraMatrix->cols == 3 in function 'cvUndistortPointsInternal'


In [27]:
cam_idx_first = 0
cam_idx_second = 2
tri_00_02 = []

for i in data_tri.index:
    _x01 = float(data_tri.loc[i]["x"+str(cam_idx_first)])
    _y01 = float(data_tri.loc[i]["y"+str(cam_idx_first)])
    _x02 = float(data_tri.loc[i]["x"+str(cam_idx_second)])
    _y02 = float(data_tri.loc[i]["y"+str(cam_idx_second)])
    tri_00_02.append(triangulatePoints(_x01,_y01,_x02,_y02,cam_idx_first,cam_idx_second))

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\calib3d\src\undistort.dispatch.cpp:284: error: (-215:Assertion failed) CV_IS_MAT(_cameraMatrix) && _cameraMatrix->rows == 3 && _cameraMatrix->cols == 3 in function 'cvUndistortPointsInternal'


In [28]:
tri_avg = (np.array(tri_00_01) + np.array(tri_00_02) + np.array(tri_01_02))/3


ValueError: operands could not be broadcast together with shapes (10,3) (0,) 

In [45]:
import glob
import numpy as np
import math
import cv2
from cv2 import aruco
import os
import matplotlib.pyplot as plt
import pandas as pd

def getVectorFromDic(idx0, idx1):
    return marker_dict[idx0]["tvec"] - marker_dict[idx1]["tvec"]

def getPositionFromDic(idx0):
    return marker_dict[idx0]["tvec"]
def getAngleFromDic(idx1, idx0, idx2):
    return getAngle(getVectorFromDic(idx2, idx0), getVectorFromDic(idx1, idx0))

def getDistanceFromDic(idx0, idx1):
    return np.linalg.norm(getVectorFromDic(idx0, idx1))

def getJointVec(arr, joint_num):
    return np.array(arr[joint_num * 4:joint_num * 4 + 3])

def normalization(vec):
    return vec / np.linalg.norm(vec)


def getSize(vec):
    return np.linalg.norm(vec)


def dotproduct(v1, v2):
    return sum((a * b) for a, b in zip(v1, v2))


def length(v):
    return math.sqrt(dotproduct(v, v))


def getAngle(v1, v2):
    return math.degrees(math.acos(dotproduct(v1, v2) / (length(v1) * length(v2))))

def vec_norm(vec):
    return vec / np.linalg.norm(vec)

def getUpVectorFromRvecs(idx):
    rtx = cv2.Rodrigues(rvecs[idx])[0]
    return rtx.T[2]

def getCenterOnImageFromDic(idx):
    return np.average(marker_dict[idx]["corner"], axis=0)

def getCenterOnImageFromDic(idx):
    return np.average(marker_dict[idx]["corner"], axis=0)

''' 
Load cameraparameters 
'''
parameter_dir = r"C:\Users\ZAIO\dev\openpose\openpose\models\cameraParameters\flir\20200810"

camera_parameters = {}
# camera_idxs = ["18284509","18284511","18284512"]
camera_idxs = ["18284509","18284511"]
parameter_types = ["CameraMatrix", "Intrinsics", "Distortion"]

for camera_idx in camera_idxs:
    fileToLoad = os.path.join(parameter_dir, camera_idx + ".xml")
    fs = cv2.FileStorage(fileToLoad, cv2.FILE_STORAGE_READ)
    _camera_parameter = {}
    for parameter_type in parameter_types:
        _camera_parameter[parameter_type] = fs.getNode(parameter_type).mat()
        camera_parameters[camera_idx] = _camera_parameter

# 3d points to image (2nd image)
extri = [0] * 3
intri = [0] * 3
dist = [0] * 3

for i in range(len(camera_idxs)):
    rotation_matrix = camera_parameters[camera_idxs[i]][parameter_types[0]][:, 0:3]
    rvec, _ = cv2.Rodrigues(rotation_matrix)
    tvec = camera_parameters[camera_idxs[i]][parameter_types[0]][:, 3]

    extri[i] = camera_parameters[camera_idxs[i]][parameter_types[0]]
    intri[i] = camera_parameters[camera_idxs[i]][parameter_types[1]]
    dist[i] = camera_parameters[camera_idxs[i]][parameter_types[2]]


In [46]:
# imagefiles_dir = r'I:\20200814_SteroTest\cam02'
# os.mkdir(os.path.join(imagefiles_dir,"processed"))
# imagefiles_dir_png = os.path.join(imagefiles_dir,"*.png")
# imagefiles = glob.glob(imagefiles_dir_png)
# print(imagefiles)

image_data = r"I:\20200814_SteroTest\000000000010_rendered.png"
camera_index = 0
mtx = camera_parameters[camera_idxs[camera_index]][parameter_types[1]]
dist = camera_parameters[camera_idxs[camera_index]][parameter_types[2]]

data_dic = {}

aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
length_of_axis = 0.09
markerLength = 0.18

frame = cv2.imread(image_data, cv2.IMREAD_UNCHANGED)
frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
parameters = aruco.DetectorParameters_create()
corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)



'''
frame_markers = aruco.drawDetectedMarkers(frame.copy(), corners, ids)

rvecs, tvecs, objPts = aruco.estimatePoseSingleMarkers(corners, markerLength, mtx, dist)
imaxis = aruco.drawDetectedMarkers(frame.copy(), corners, ids)
if rvecs is not None and len(rvecs) == 4:
    for i in range(len(rvecs)):
        imaxis = aruco.drawAxis(imaxis, mtx, dist, rvecs[i], tvecs[i], length_of_axis)

    # Marker data to dictionary
    marker_dict = {}

    for idx, marker_index in enumerate(ids):
        marker_index = marker_index[0]
        _vecs = {}
        _vecs["rvec"] = rvecs[idx][0]
        _vecs["tvec"] = tvecs[idx][0]
        _vecs["corner"] = corners[idx][0]
        marker_dict[marker_index] = _vecs
    # print("tvecs and rvecs for each marker\n")
    # pp.pprint(marker_dict)

    plt.figure()
    plt.figure(figsize=(15, 15))

    data_array = []
    for idx0, idx1 in [[0, 1], [1, 2], [2, 3]]:
        cen_x, cen_y = np.average([getCenterOnImageFromDic(idx0), getCenterOnImageFromDic(idx1)], axis=0)
        distance = getDistanceFromDic(idx0, idx1)
        text = plt.text(int(cen_x), int(cen_y), "{:.3f}m".format(distance), fontsize=9, color="red")
        data_array.append(distance)
        text.set_bbox(dict(facecolor='white', alpha=1, edgecolor='white'))
    plt.imshow(imaxis)
    dir, file = os.path.split(image_data)
    new_dir = os.path.join(dir, "processed")
    print(os.path.join(new_dir, file))
    plt.savefig(os.path.join(new_dir, file), dpi=300)
    data_dic[file] = data_array
    # plt.show()
'''

'\nframe_markers = aruco.drawDetectedMarkers(frame.copy(), corners, ids)\n\nrvecs, tvecs, objPts = aruco.estimatePoseSingleMarkers(corners, markerLength, mtx, dist)\nimaxis = aruco.drawDetectedMarkers(frame.copy(), corners, ids)\nif rvecs is not None and len(rvecs) == 4:\n    for i in range(len(rvecs)):\n        imaxis = aruco.drawAxis(imaxis, mtx, dist, rvecs[i], tvecs[i], length_of_axis)\n\n    # Marker data to dictionary\n    marker_dict = {}\n\n    for idx, marker_index in enumerate(ids):\n        marker_index = marker_index[0]\n        _vecs = {}\n        _vecs["rvec"] = rvecs[idx][0]\n        _vecs["tvec"] = tvecs[idx][0]\n        _vecs["corner"] = corners[idx][0]\n        marker_dict[marker_index] = _vecs\n    # print("tvecs and rvecs for each marker\n")\n    # pp.pprint(marker_dict)\n\n    plt.figure()\n    plt.figure(figsize=(15, 15))\n\n    data_array = []\n    for idx0, idx1 in [[0, 1], [1, 2], [2, 3]]:\n        cen_x, cen_y = np.average([getCenterOnImageFromDic(idx0), getCe

In [47]:
marker_dict00 = {}
marker_dict01 = {}

In [48]:
image_data = r"I:\20200814_SteroTest\000000000010_rendered.png"
camera_index = 0
mtx = camera_parameters[camera_idxs[camera_index]][parameter_types[1]]
dist = camera_parameters[camera_idxs[camera_index]][parameter_types[2]]

data_dic = {}

aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
length_of_axis = 0.09
markerLength = 0.18

frame = cv2.imread(image_data, cv2.IMREAD_UNCHANGED)
frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
parameters = aruco.DetectorParameters_create()
corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

for idx, marker_index in enumerate(ids):
    marker_index = marker_index[0]
    _vecs = {}
    _vecs["corner"] = corners[idx][0]
    marker_dict00[marker_index] = _vecs

In [49]:
image_data = r"I:\20200814_SteroTest\000000000010_rendered_1.png"
camera_index = 1
mtx = camera_parameters[camera_idxs[camera_index]][parameter_types[1]]
dist = camera_parameters[camera_idxs[camera_index]][parameter_types[2]]

data_dic = {}

aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
length_of_axis = 0.09
markerLength = 0.18

frame = cv2.imread(image_data, cv2.IMREAD_UNCHANGED)
frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
parameters = aruco.DetectorParameters_create()
corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
for idx, marker_index in enumerate(ids):
    marker_index = marker_index[0]
    _vecs = {}
    _vecs["corner"] = corners[idx][0]
    marker_dict01[marker_index] = _vecs


In [50]:
marker_dict00[0]["corner"]
marker_dict01[0]["corner"]

array([[207., 664.],
       [319., 672.],
       [309., 788.],
       [193., 778.]], dtype=float32)

In [51]:

cam_idx_first = 0
cam_idx_second = 1
tri_00_01 = []

_x01,_y01 = marker_dict00[0]["corner"][0]
_x02, _y02= marker_dict01[0]["corner"][0]
tri_00_01.append(triangulatePoints(_x01,_y01,_x02,_y02,cam_idx_first,cam_idx_second))
tri_00_01

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\calib3d\src\undistort.dispatch.cpp:297: error: (-215:Assertion failed) CV_IS_MAT(_distCoeffs) && (_distCoeffs->rows == 1 || _distCoeffs->cols == 1) && (_distCoeffs->rows*_distCoeffs->cols == 4 || _distCoeffs->rows*_distCoeffs->cols == 5 || _distCoeffs->rows*_distCoeffs->cols == 8 || _distCoeffs->rows*_distCoeffs->cols == 12 || _distCoeffs->rows*_distCoeffs->cols == 14) in function 'cvUndistortPointsInternal'
